In [21]:
import pandas as pd
import nltk
import emoji
import re
from IPython.display import display

pd.set_option('display.max_columns', None) # or 1000.

In [22]:
df1 = pd.read_json('formulae-snscrape.json', lines=True)
df2 = pd.read_csv('formula_e.csv')
df3 = pd.read_json('formulae-snscrape2.json', lines=True)
print(df1.info())
print(df2.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 28 columns):
 #   Column            Non-Null Count  Dtype              
---  ------            --------------  -----              
 0   _type             10000 non-null  object             
 1   url               10000 non-null  object             
 2   date              10000 non-null  datetime64[ns, UTC]
 3   content           10000 non-null  object             
 4   renderedContent   10000 non-null  object             
 5   id                10000 non-null  int64              
 6   user              10000 non-null  object             
 7   replyCount        10000 non-null  int64              
 8   retweetCount      10000 non-null  int64              
 9   likeCount         10000 non-null  int64              
 10  quoteCount        10000 non-null  int64              
 11  conversationId    10000 non-null  int64              
 12  lang              10000 non-null  object             
 13  so

In [23]:
df2 = df2.drop('Unnamed: 0', axis=1)
df2.columns

Index(['id', 'conversation_id', 'created_at', 'date', 'time', 'timezone',
       'user_id', 'username', 'name', 'place', 'tweet', 'language', 'mentions',
       'urls', 'photos', 'replies_count', 'retweets_count', 'likes_count',
       'hashtags', 'cashtags', 'link', 'retweet', 'quote_url', 'video',
       'thumbnail', 'near', 'geo', 'source', 'user_rt_id', 'user_rt',
       'retweet_id', 'reply_to', 'retweet_date', 'translate', 'trans_src',
       'trans_dest'],
      dtype='object')

In [24]:
df2_indo = df2[df2.language == 'in']
df2_indo.tweet

0       Gembong-PDIP Curiga Anies Paksa ASN DKI Beli T...
1       KPK tegaskan tak main-main usut dugaan korupsi...
2       Pengaspalan sirkuit Formula E Jakarta di kawas...
3       Dia, melabali pengkritik sebagai pembebci...!!...
4       KPK Harus Naikkan Status Kasus Formula E ke Pe...
                              ...                        
7159    Wakil Ketua Komisi E DPRD DKI Anggara Wicitra ...
7160    @AlfathMauzy @f1speed_indo "Wow ini lah tampak...
7162      @asumsico Kirain formula e yg mau ditambahin jg
7163    @danugraha54 @YZhireng @haluanghabang @Dennysi...
7166    Kapan Tiket Formula E Mulai Dijual? Begini Jaw...
Name: tweet, Length: 2960, dtype: object

In [25]:
df1_indo = df1[df1.lang == 'in']
df1_indo['content']

0       Dalam dua bulan Jakarta akan menjadi tuan ruma...
1       KPK tegaskan tak main-main usut dugaan korupsi...
2       Pengaspalan sirkuit Formula E Jakarta di kawas...
3       https://t.co/CJ0pw6Hu0O\nKlo dilaporin dalangn...
4       @Adi_8002 Kadrun mau demo" di jakarta, sementa...
                              ...                        
9995    @umar_hasibuan75 @KPK_RI @paijodirajo @yudihar...
9996    (24)Coba biaya Formula E yang sampai triliunan...
9997    Loh, ada apa ini?\nJakpro kan  penyelenggara F...
9998    Direktur Eksekutif LKAB Rudi S Kamri mengataka...
9999    Loh, ada apa ini?\nJakpro kan  penyelenggara F...
Name: content, Length: 9561, dtype: object

In [26]:
df3_indo = df3[df3.lang == 'in']
df3_indo['content']

0        Yang jelas, dua fraksi DPRD DKI Jakarta dikaba...
1        Yang sudah nunggu penjualan tiket Formula E Ja...
2        Sebelumnya, wacana hak interpelasi Formula E i...
3        Ketua DPRD DKI Jakarta Prasetyo Edi mempersila...
4        "Alhamdulillah, berhasil! Jakarta akan menjadi...
                               ...                        
28883    Kemenkes: Tak ada bakteri E Sakazakii pada sus...
28884    Kemenkes: Tak ada bakteri E Sakazakii pada sus...
28885    Kemenkes: Tak ada bakteri E Sakazakii pada sus...
28886    Kemenkes: Tak ada bakteri E Sakazakii pada sus...
28887    Kemenkes: Tak Ada Bakteri E Sakazakii pada 47 ...
Name: content, Length: 26026, dtype: object

In [27]:
tweets_df = pd.DataFrame()

tweets_df['tweets'] = pd.concat([df1_indo.content, df2_indo.tweet, df3_indo.content], ignore_index=True)
print(tweets_df.shape)
tweets_df

(38547, 1)


,tweets
0,Dalam dua bulan Jakarta akan menjadi tuan ruma...
1,KPK tegaskan tak main-main usut dugaan korupsi...
2,Pengaspalan sirkuit Formula E Jakarta di kawas...
3,https://t.co/CJ0pw6Hu0O\nKlo dilaporin dalangn...
4,"@Adi_8002 Kadrun mau demo"" di jakarta, sementa..."
...,...
38542,Kemenkes: Tak ada bakteri E Sakazakii pada sus...
38543,Kemenkes: Tak ada bakteri E Sakazakii pada sus...
38544,Kemenkes: Tak ada bakteri E Sakazakii pada sus...
38545,Kemenkes: Tak ada bakteri E Sakazakii pada sus...


In [28]:
tweets_df = tweets_df.drop_duplicates(ignore_index=True)
tweets_df

,tweets
0,Dalam dua bulan Jakarta akan menjadi tuan ruma...
1,KPK tegaskan tak main-main usut dugaan korupsi...
2,Pengaspalan sirkuit Formula E Jakarta di kawas...
3,https://t.co/CJ0pw6Hu0O\nKlo dilaporin dalangn...
4,"@Adi_8002 Kadrun mau demo"" di jakarta, sementa..."
...,...
28083,Kemenkes: Tak Ada Bakteri E Sakazakii pada 47 ...
28084,Kemenkes: Tak ada bakteri E Sakazakii pada sus...
28085,Kemenkes: Tak ada bakteri E Sakazakii pada sus...
28086,Kemenkes: Tak ada bakteri E Sakazakii pada sus...


In [29]:
def clean_data(series):
    clean_tweets = []
    for tweet in series:
        tweet = re.sub("@[A-Za-z0-9]\w+","",tweet) #Remove @ sign
        tweet = re.sub(r"(?:\@|http?\://|https?\://|www)\S+", "", tweet) #Remove http links
        # tweet = emoji.replace_emoji(tweet, '') # Remove emoji
        # tweet = tweet.replace("#", "").replace("_", " ") #Remove hashtag sign but keep the text
        tweet = re.sub(r"\n", "", tweet)
        tweet = tweet.strip()
        clean_tweets.append(tweet)
    return clean_tweets

tweets_df['clean_tweets'] = clean_data(tweets_df.tweets)
tweets_df = tweets_df.drop_duplicates(ignore_index=True)
tweets_df

/tmp/ipykernel_564520/3230039042.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tweets_df['clean_tweets'] = clean_data(tweets_df.tweets)


,tweets,clean_tweets
0,Dalam dua bulan Jakarta akan menjadi tuan ruma...,Dalam dua bulan Jakarta akan menjadi tuan ruma...
1,KPK tegaskan tak main-main usut dugaan korupsi...,KPK tegaskan tak main-main usut dugaan korupsi...
2,Pengaspalan sirkuit Formula E Jakarta di kawas...,Pengaspalan sirkuit Formula E Jakarta di kawas...
3,https://t.co/CJ0pw6Hu0O\nKlo dilaporin dalangn...,"Klo dilaporin dalangnya siapa, apa brani ngus..."
4,"@Adi_8002 Kadrun mau demo"" di jakarta, sementa...","Kadrun mau demo"" di jakarta, sementara gubernu..."
...,...,...
28083,Kemenkes: Tak Ada Bakteri E Sakazakii pada 47 ...,Kemenkes: Tak Ada Bakteri E Sakazakii pada 47 ...
28084,Kemenkes: Tak ada bakteri E Sakazakii pada sus...,Kemenkes: Tak ada bakteri E Sakazakii pada sus...
28085,Kemenkes: Tak ada bakteri E Sakazakii pada sus...,Kemenkes: Tak ada bakteri E Sakazakii pada sus...
28086,Kemenkes: Tak ada bakteri E Sakazakii pada sus...,Kemenkes: Tak ada bakteri E Sakazakii pada sus...


In [30]:
print(tweets_df.tweets[5], '\n')
print(tweets_df.clean_tweets[5])

OPEN PROMOTE
🇮🇩🇮🇩🇮🇩Ketika Followers suaranya dipadukan maka kebersamaan menjadi pilihan❤❤❤💞💞🌹🌹
#UjarPolitik
#DuniaFolback 
Selancarlah bersamaku🏄🏻
@dionedone 
🏎🏎Akankah Formula E menjadi ajang pilihan warga Jakarta?👥👥👥

@el_azp 
@0m_Gatot 
@K444Dhek 
@IBRAHIMCruise https://t.co/BeE5PE0Cue 

OPEN PROMOTE🇮🇩🇮🇩🇮🇩Ketika Followers suaranya dipadukan maka kebersamaan menjadi pilihan❤❤❤💞💞🌹🌹#UjarPolitik#DuniaFolback Selancarlah bersamaku🏄🏻 🏎🏎Akankah Formula E menjadi ajang pilihan warga Jakarta?👥👥👥


In [31]:
tweets_df.to_csv('unlabelled_data.csv')

In [45]:
# clean_tweets_df = 
clean_tweets = pd.DataFrame()
clean_tweets['tweets'] = tweets_df[['clean_tweets']]
clean_tweets = clean_tweets.drop_duplicates(ignore_index=True)

In [43]:
clean_tweets.to_csv('unlabelled_data.csv')

In [46]:
clean_tweets[:10].to_csv('subset_data.csv')